In [33]:
from statsmodels.tsa.api import VAR, AutoReg
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA
from typing import Dict, List, Tuple
from joblib import Parallel, delayed
from ipca import InstrumentedPCA
import statsmodels.api as sm
from scipy.stats import norm
import pandas as pd
import numpy as np
import pickle
import random
import time


In [34]:
def formIndexCrosswalk(in_df: pd.DataFrame) -> pd.DataFrame:
    """
    Form a crosswalk dataframe to new indices that are integer indices for use with IPCA package.
    """
    cross_df = in_df[['date', 'asset']].copy()
    cross_df['time'] = cross_df['date'].factorize()[0] + 1
    cross_df['asset_num'] = cross_df['asset'].factorize()[0] + 1
    return cross_df

def normalize_column(df, column_name):
    """ 
    Cross sectionally normalize column by values in 'time' column.
    """
    def normalize_within_date_group(column_name, group):
        # Calculate the number of assets for this date
        n = group.shape[0]
        
        # Add random noise to the values to ensure unique ranks
        noise = np.random.uniform(-1e-6, 1e-6, size=n)
        group[column_name] += noise
        
        # Rank the values, divide by the number of assets, and subtract 0.5
        group[column_name] = group[column_name].rank() / n - 0.5
        return group
    
    # Apply the normalization to a specific column within each date group
    return df.groupby('time', group_keys=False).apply(lambda group: normalize_within_date_group(column_name, group))

def subsetAndNormalizeColumns(in_df: pd.DataFrame, lhs_col: str, p: int) -> pd.DataFrame:
    """
    Subset the weekly panel to relevant columns for fitting IPCA and 
        normalizes the characteristic columns using cross-sectional ranking.
    
    This function linearly spaces the values in the characteristic columns within each date
    to the range [-0.5, 0.5]. The values are ranked, divided by the number of assets for
    that date, and subtracted by 0.5.

    Args:
        in_df (pd.DataFrame): weekly panel data with all the columns.
        lhs_col (str): name of the left hand side column.
        p (int): number of characteristics to use in the simulation.
    
    Returns: (pd.DataFrame): weekly panl with IPCA asset and time columns
        lhs col and the p characteristics.
    """
    # Form a copy
    df = in_df.copy()

    # Determine characteristic columns
    char_cols = [col for col in list(df.columns.values) 
                    if ('char_' in col) & ~('char_industry_' in col) 
                    & ~('char_asset_usage' in col)]
    for col in ['char_pow', 'char_pos', 'char_ico_price', 'char_ico']:
        char_cols.remove(col)

    # Randomly obtain p of the characteristics.
    random.seed(42)
    selected_columns = random.sample(char_cols, p)

    # Subset to relevant columns
    df = df[['time', 'asset_num', lhs_col]+selected_columns].copy()

    # Loop over the specified columns to normalize
    for column_name in selected_columns:
        df = normalize_column(df, column_name)
    
    # Random
    return df, selected_columns

def fitIPCAInPanelForEstimatedFactorsAndLoadings(
    in_df: pd.DataFrame, lhs_col: str, p: int, k: int, fit_alpha: bool, num_cpus: int):
    """
    Generates an AR(1) process for a given set of parameters and length T using numpy.
    
    Args:
        in_df (pd.DataFrame): weekly panel data with all the columns.
        lhs_col (str): name of the left hand side column.
        p (int): number of characteristics to use in the simulation.
        k (int): number of factors.
        fit_alpha (bool): whether to include an intercept alpha term.
        num_cpus (int): number of processors to use when working in parallel.
        
    Returns: (Tuple) numpy array of factors of num of weeks in panel by k; 
                numpy array of loadings of p * k; 
                and a float of the empirical r^2_total.
    """
    # Form a copy of the data
    df = in_df.copy()

    # form index crosswalk, in case needed
    cross_df = formIndexCrosswalk(df)
    df = df.merge(cross_df, on=['date', 'asset'], how='inner', validate='one_to_one')

    # Subset columns
    df, selected_columns = subsetAndNormalizeColumns(df, lhs_col, p)

    # Form list of rhs char columns
    char_cols = [col for col in list(df.columns.values) if 'char_' in col]
    assert(len(char_cols)==p)

    # Form datasets to fit ipca
    df = df.sort_values(by=['time', 'asset_num'], ignore_index=True)
    Y  = df[['time', 'asset_num', lhs_col]].copy()
    Y[lhs_col] = Y[lhs_col].astype('float64')
    Y = Y.set_index(keys=['asset_num', 'time'], verify_integrity=True)
    Y = Y.squeeze()
    X = df[['time', 'asset_num']+char_cols].copy()
    X = X.set_index(keys=['asset_num', 'time'], verify_integrity=True)
    X = X.astype('float64')

    # Fit
    ipca = InstrumentedPCA(n_factors=k, intercept=fit_alpha, n_jobs=num_cpus)
    ipca = ipca.fit(X=X, y=Y, data_type='panel')

    # Form factors and loadings to return
    estimates = ipca.get_factors()
    loadings  = estimates[0]
    factors   = estimates[1].transpose()

    # Obtain the R^2_total
    yhats = ipca.predict(X, data_type='panel')
    r2_total = 1 - np.mean(np.square(Y - yhats))/np.mean(np.square(Y))

    return factors, loadings, r2_total, selected_columns


In [35]:
def fitVAR1(matrix: np.ndarray) -> (np.ndarray, np.ndarray):
    """
    Fit a VAR(1) model to the given matrix.

    Parameters:
    - matrix (ndarray): Input data matrix with dimensions T x k.

    Returns:
    - tuple: Intercept vector (mu) and coefficients matrix (phi).
    """
    if matrix.shape[1] == 1:
        model = AutoReg(matrix.squeeze(), lags=1)
        model_fitted = model.fit()
        mu = model_fitted.params[0]  # intercept
        phi = model_fitted.params[1]  # coefficients matrix
    else:
        model = VAR(matrix)
        model_fitted = model.fit(1)
        mu = model_fitted.params[0]  # intercept
        phi = model_fitted.params[1:]  # coefficients matrix
    return mu, phi

def simulateVAR1(mu: np.ndarray,
    phi: np.ndarray,
    mean: np.ndarray,
    variance: np.ndarray,
    n_steps: int,
    k: int,
    s: int) -> np.ndarray:
    """
    Simulate new realizations from a fitted VAR(1) model.

    Parameters:
    - mu (ndarray): Intercept vector from the fitted model.
    - phi (ndarray): Coefficients matrix from the fitted model.
    - mean (ndarray): Mean of the normal innovations.
    - variance (ndarray): Variance of the normal innovations.
    - n_steps (int): Number of time steps to simulate.
    - k (int): Number of variables.
    - s (int): seed number to pass to produce variation.

    Returns:
    - ndarray: Simulated data of dimensions n_steps x k.
    """
    # Set buffer to throw away first obs
    buffer = int(n_steps/4)

    # Set seed for replicability
    np.random.seed(s)

    # Initialize array to store simulated values
    simulated_data = np.zeros((n_steps+buffer, k))

    # Set initial value
    if k > 1:
        simulated_data[0] = np.random.multivariate_normal(mean, np.diag(variance), 1)
    else:
        simulated_data[0] = np.random.normal(mean, np.sqrt(variance), 1)

    # Run simulation
    for t in range(1, n_steps+buffer):
        # form factors for a single or multivariate AR model
        if k > 1:
            epsilon_t = np.random.multivariate_normal(np.zeros(k), np.diag(variance), 1)
            simulated_data[t] = mu + np.matmul(phi, simulated_data[t-1]) + epsilon_t
        else:
            epsilon_t = np.random.normal(0, np.sqrt(variance), 1)
            simulated_data[t] = mu + phi * simulated_data[t-1] + epsilon_t
        
    return simulated_data[buffer:,:]

def simulateFactors(ipca_factors: np.ndarray, T: int, S: int, k: int, sigma_f_2: float) -> np.ndarray:
    """ Use VAR(1) model to simulate the requested factor matrix from empirical data in ipca_factors.

    Args:
        ipca_factors (np.ndarray): matrix of factors of dimensions T by number of factors.
        T (int): number of time periods in the simulation.
        S (int): number of simulations to run.
        k (int): number of factors.
        sigma_f_2 (float): variance of normal innovations in VAR(1) model for factors.

    Returns:
        np.ndarray: sim'ed factors of dimensions T x number of covar_cols in `factors' x S.
    """
    # Create objects
    new_factors = np.zeros((T, k, S), dtype=np.float64)

    # Fit VAR(1) on the fitted latent factors from empirical data.
    mu, phi = fitVAR1(ipca_factors)

    # Set VAR(1) simulation params
    if k > 1:
        innovations_mean = np.zeros(k)
        innovations_var = np.repeat(sigma_f_2, k)
    else:
        innovations_mean = 0
        innovations_var = sigma_f_2

    # Simulate new factors using the VAR(1) model with normal innovations for given number of sims
    for s in range(S):
        new_factors[:,:,s] = simulateVAR1(mu, phi, innovations_mean, innovations_var, T, k, s)

    return new_factors

def simulateCharacteristics(chars_df: pd.DataFrame, S: int, T: int, N: int, p: int, sigma_z_2: float, num_cpus: int):
    """ Use VAR(1) model to simulate the characteristics a la Kelly et al 2020 IPCA Theory paper.

    Args:
        chars_df (pd.DataFrame): contains characteristic data for all time periods and assets in empirical data.
        N (int): number of assets in the simulation.
        S (int): number of simulations to run.
        T (int): number of time periods in the simulation.
        p (int): number of characteristics used in the simulation.
        sigma_z_2 (float): variance of normal innovations in VAR(1) model for characteristics.
        num_cpus (int): number of cpus to use to run this simulation in parallel.

    Returns:
        np.ndarray: sim'ed characteristics of dimensions T*N x p x S.
    """
    # List of assets to loop over
    assets = list(np.unique(chars_df.asset.values))

    # Remove assets to use to generate simulated characteristics if less than 200 time periods of data
    raw_assets = assets.copy()
    for asset in raw_assets:
        num_obs = chars_df[chars_df.asset==asset].shape[0]
        if num_obs < 200:
            assets.remove(asset)

    # If number of assets we have is more than we need, then downsample; else, upsample.
    if len(assets) >= N:
        assets = random.sample(assets, N)
    else:
        diff = N - len(assets)
        assets += random.choices(assets, k=diff)
        assets = sorted(assets, key=lambda x: random.random())
    assert(len(assets)==N)

    # Define function to loop over all the assets
    def loopOverAsset(i):
        # Grab asset name
        asset = assets[i]

        # Subset to asset data
        asset_df = chars_df[chars_df.asset==asset].copy()

        # Obtain the matrix of characteristics
        chars_mat = asset_df.drop(['date', 'asset'], axis=1).values

        # Normalize the data
        column_means = np.mean(chars_mat, axis=0)
        column_stds = np.std(chars_mat, axis=0)
        chars_mat = (chars_mat - column_means) / column_stds

        # Calculate the time series average of each characteristic
        char_means = np.mean(chars_mat, axis=0)

        # Calcuate the VAR(1) on the matrix of characteristics
        mu, phi = fitVAR1(chars_mat)

        # Set VAR(1) simulation params
        innovations_means = random.choices(char_means, k=p)
        innovations_var = np.repeat(sigma_z_2, p)

        # Simulate new instruments using the VAR(1) model with normal innovations for given number of sims
        sim_chars = []
        for s in range(S):
            sim_chars.append(simulateVAR1(mu, phi, innovations_means, innovations_var, T, p, s))

        return np.stack(sim_chars, axis=-1)

    # Generate characteristics for given number of assets
    sim_chars_per_asset = Parallel(n_jobs=num_cpus)(delayed(loopOverAsset)(i) for i in range(len(assets)))

    # Reshape and return results
    Z = np.stack(sim_chars_per_asset, axis=-1).transpose(0, 3, 1, 2).reshape(T*N, p, S)

    return Z


In [36]:
def DGP(T: int, N: int, k: int, p: int, sparse_s: int, S: int, eta: np.ndarray,
    sigma_f_2: float, sigma_z_2: float, sigma_g_2: float, sigma_e_2: float,
    ipca_factors: np.ndarray, ipca_loadings: np.ndarray, r2_total_emp: float,
    selected_columns: List[str], panel_df: pd.DataFrame, num_cpus: int) -> tuple:
    ''' Generate random variables and parameters for simulation.

    Args: 
        T (int): number of time periods.
        N (int): number of observational units per time period.
        k (int): dimensionality of latent factors.
        p (int): dimensionality of the covariates.
        sparse_s (int): sparsity index of factor loadings.
        S (int): number of simulations.
        eta (np.ndarray): parameter mapping latent factors to observable factor.
        sigma_f_2 (float): variance of errors in latent factor VAR(1) sim. 
        sigma_z_2 (float): variance of errors in characteristic PVAR(1) sim. 
        sigma_g_2 (float): variance of errors in observable factor model.
        sigma_e_2 (float): variance of idiosyncratic errors.
        ipca_factors (np.ndarray): matrix of factors of dimensions T by number of factors.
        ipca_loadings (np.ndarray): matrix of loadings of dimensions p by k estimated from data.
        r2_total_emp (float): the empirical r^2_total.
        selected_columns (List[str]): columns selected from empirical panel from simulation characteristics.
        panel_df (pd.DataFrame): panel of empirical data.
        num_cpus (int): number of CPUs to use for parallel processing.
    
    Returns: 
        (tuple):
            - R (np.ndarray): outcome/returns ndarray of dimensions T*N by 1 by S.
            - Gamma_beta (np.ndarray): true loading on latent factors of dim p by k.
            - F (np.ndarray): true latent factors of dimensions T by k by S.
            - G (np.ndarray): true observable factor of dimensions T by 1 by S.
            - Z (np.ndarray): covariates of dimensions T*N by p by S.
            - Gamma (np.ndarray): true time series averages of latent factors k by S.
            - e (np.ndarray): true idiosyncratic error of dimensions T*N, 1, S.
            - e_g (np.ndarray): true error in ob factor model of dimensions T, 1, S.
    '''
    # Initialize data objects 
    R     = np.zeros((T*N, 1, S), dtype=float)
    G     = np.zeros((T, 1, S), dtype=float)
    Gamma = np.zeros((k, 1, S), dtype=float)
    e     = np.zeros((T*N, 1, S), dtype=float)
    e_g   = np.zeros((T, 1, S), dtype=float)

    # Form Gamma Beta by soft thresholding the empirically estimated loadings to leave only
    #     sparse_s rows nonzero and re order from high to low by row l1 norm
    emp_loadings_row_l1_norm    = np.sum(np.abs(ipca_loadings), axis=1)
    quantile                    = np.quantile(emp_loadings_row_l1_norm, q=1-sparse_s/p)
    sparse_mask                 = emp_loadings_row_l1_norm > quantile
    assert(sparse_s == np.sum(sparse_mask))
    Gamma_beta = ipca_loadings.copy()
    Gamma_beta[~sparse_mask, :] = 0
    indices_ordered_by_l1_norm  = np.argsort(emp_loadings_row_l1_norm)[::-1]
    Gamma_beta                  = Gamma_beta[indices_ordered_by_l1_norm]

    # Simulate factors
    F = simulateFactors(ipca_factors, T, S, k, sigma_f_2)

    # Simulate characteristics
    Z = simulateCharacteristics(panel_df[['date', 'asset']+selected_columns], S, T, N, p, sigma_z_2, num_cpus)

    # generate data for each simulation
    for s in range(S):
        # set seed for numpy and random packages for replicable data
        np.random.seed(int(42*s))

        # form idiosyncratic errors
        e[:,:,s] = np.random.multivariate_normal([0], [[sigma_e_2]], size=T*N)

        # Form returns for each time period
        for t in range(T):
            # Form start and end index for indexing T*N vectors
            start_index = int(t*N)
            end_index   = int((t+1)*N)

            # Form returns
            R[start_index:end_index, :, s] = ((Z[start_index:end_index,:,s] @ Gamma_beta @ F[t,:,s]).reshape(-1,1) 
                + e[start_index:end_index,:,s])

        # form observable factor and time series mean of factors
        e_g[:,:,s]   = np.random.multivariate_normal([0], [[sigma_g_2]], size=T)
        Gamma[:,:,s] = F[:,:,s].mean(axis=0).reshape(-1,1)
        V            = F[:,:,s] - Gamma[:,:,s].reshape(-1)
        G[:,:,s]     = V.dot(eta).reshape(-1,1) + e_g[:,:,s]

    return R, Gamma_beta, F, G, Z, Gamma, e, e_g


In [37]:
def runLasso(Y: np.ndarray, X: np.ndarray, penalty: float) -> np.ndarray:
    ''' Runs lasso of Y on X with given penalty param to return fitted coefs.

    Args: 
        X (np.ndarray): RHS variables with rows of obs and covar_cols of covars.
                        These data include a constant but have yet to be
                        normalized for lasso.
        Y (np.ndarray): LHS variable with rows of obs and single column.
        penalty (float): real-valued scalar on L1 penalty in Lasso.

    Returns:
        beta_hat (np.ndarray): vector of fitted coefficients; note: these
                                must be used on normalized RHS variables.
    '''
    # normalize RHS
    muhat  = np.mean(X, axis = 0)
    stdhat = np.std(X, axis = 0)+np.min(X)/1e3 # NOTE: incase of no variation wont divide by 0
    Xtilde = np.divide(np.subtract(X, muhat), stdhat)

    # perform lasso
    lasso = Lasso(alpha = penalty)
    lasso.fit(Xtilde, Y)

    # return fitted coefficients
    return lasso.coef_

def calcPenaltyBCCH(Y: np.ndarray, X: np.ndarray, c: float) -> float:
    ''' This function applies Belloni, Chen, Chernozhukov, Hansen 2012 ECMA
        closed-form solution for selecting Lasso penalty parmaeter.

    Args: 
        X (np.ndarray): RHS variables with rows of obs and covar_cols of covars.
                        These data include a constant but have yet to be
                        normalized for lasso.
        Y (np.ndarray): LHS variable with rows of obs and single column.
        c (float):    scalar constant from theory; usually ~1.

    Returns:
        penalty (float): BCCH penalty parameter.
    '''
    # Bickel Ritov Tsybakov constant parameter selection
    a = 0.1

    # calc pilot penalty parameter
    N = X.shape[0]
    p = X.shape[1]
    max_moment_xy = np.max(np.mean((X**2)*(Y**2), axis =0)**0.5) 
    penalty_pilot = 2*c*norm.ppf(1-a/(2*p))*max_moment_xy/np.sqrt(N)

    # run lasso with pilot penalty parameter
    beta_hat = runLasso(Y, X, penalty_pilot)
    #assert(~np.isclose(0, np.sum(np.abs(beta_hat)), rtol=1e-8, atol=1e-8)),('Pilot penalty kills all coefs. Scale down c?')

    # set BCCH penalty parameter
    residuals = Y - np.matmul(X, beta_hat).reshape(-1,1)
    max_moment_xepi = np.max(np.mean((X**2)*(residuals**2), axis =0)**0.5) 
    penalty = 2*c*norm.ppf(1-a/(2*p))*max_moment_xepi/np.sqrt(N)

    return penalty

def runOLS(Y: np.ndarray, X: np.ndarray) -> np.ndarray:
    ''' Runs OLS of Y on X to return fitted coefficients.

    Args: 
        X (np.ndarray): RHS--assumes contains constant--with rows of obs and covar_cols of covars.
        Y (np.ndarray): LHS variable with rows of obs and single column.

    Returns:
        beta_hat (np.ndarray): vector of fitted coefficients.
    '''
    return np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)),
                        np.matmul(np.transpose(X), Y))

def runDoubleSelectionLasso(Y: np.ndarray, D: np.ndarray, X: np.ndarray, c: float,
    selected_prct_upper: float=0.5, selected_prct_lower: float=0.05) -> float:
    ''' Runs Double Selection Lasso from Belloni et al (2014).

    Args: 
        Y (np.ndarray): LHS variable with rows of obs and single column.
        D (np.ndarray): RHS target variable with rows of obs and single column.
        X (np.ndarray): RHS controls with rows of obs and p cols of characteristics.
        c (float): scalar constant from theory; usually ~1.
        selected_prct_upper (float): upper bound on number of columns selected.
        selected_prct_lower (float): lower bound on number of columns selected.
    
    Returns:
        alpha_hat (float): estimated target coefficient.
    '''
    # initialize a percent selected outside range
    selected_prct_cols = 1

    while ((selected_prct_cols > selected_prct_upper) 
        | (selected_prct_cols < selected_prct_lower)):
        # update scalar constant
        if (selected_prct_cols > selected_prct_upper):
            c = 1.05*c
        else:
            c = 0.95*c

        # lasso of Y on D and X to select elements of X, I_1_hat
        X_all = np.hstack((D,X))
        beta_hat_1 = runLasso(Y, X_all, penalty=calcPenaltyBCCH(Y, X_all, c=c))

        # lasso of D on X to select elements of X, I_2_hat
        beta_hat_2 = runLasso(D, X, penalty=calcPenaltyBCCH(D, X, c=c))

        # form union of I_1_hat and I_2_hat
        i_1_hat = list(np.nonzero(beta_hat_1)[0] -1 ) # NOTE: subtracting 1 as we added the treatment var to RHS
        if -1 in i_1_hat: i_1_hat.remove(-1) # remove treatment variable if it was included
        i_2_hat = list(np.nonzero(beta_hat_2)[0])
        i_hat   = list(set(i_1_hat).union(set(i_2_hat)))

        # update percent of columns that were selected
        selected_prct_cols = len(i_hat) / X.shape[1]

    # OLS of Y on D plus included Xs
    X_sel    = X[:,i_hat]
    X_all    = np.hstack((D, X_sel))
    beta_hat = runOLS(Y, X_all)
    alpha_hat = beta_hat[0,0]

    # return target parameter on D
    return alpha_hat

def fitBaiPCA(
    matrix: np.ndarray, T: int, k: int, p: int
    ) -> Tuple[np.ndarray, np.ndarray]:
    # Calculate the scaling factor
    scaling_factor = 1 / (T * p)

    # Form the target, symmetric, positive semi-definite matrix
    target_matrix = scaling_factor * (matrix @ matrix.T)

    # Calculate eigenvalues and vectors
    eigenvalues, eigenvectors = np.linalg.eigh(target_matrix)

    # Calculate factors and loadings
    factors = np.sqrt(T) * eigenvectors[:, -k:][:, ::-1]
    loadings = (matrix.T @ factors) / T

    # Confirm factors are scaled appropriately
    identity = (factors.T @ factors) / T
    assert(np.isclose(k, np.sum(np.abs(np.diagonal(identity)))))

    return factors, loadings

def softThresholdCols(matrix, sparse_prct=0.2):
    """ Implement ell_1 soft thresholding across colums of the given matrix. """
    dim1 = matrix.shape[0]
    ell_1_norm_cols = np.sum(np.abs(matrix), axis=0)
    lmbd = np.quantile(ell_1_norm_cols, 1-0.2)
    col_mask = 1*(ell_1_norm_cols > lmbd)
    mat_mask = np.tile(col_mask, (dim1, 1))
    return matrix*mat_mask

def softThresholdRows(matrix, sparse_prct=0.2):
    """ Implement ell_1 soft thresholding across rows of the given matrix. """
    dim2 = matrix.shape[1]
    ell_1_norm_rows = np.sum(np.abs(matrix), axis=1)
    lmbd = np.quantile(ell_1_norm_rows, 1-sparse_prct)
    row_mask = 1*(ell_1_norm_rows > lmbd)
    mat_mask = np.repeat(row_mask, dim2).reshape(-1, dim2)
    return matrix*mat_mask

def fitDSLFM(R: np.ndarray, Z: np.ndarray,
    T: int, N: int, k: int, p: int, c: float, num_cpus: int) -> np.ndarray:
    ''' This function performs the DSLFM estimation procedure.

    Args: 
        R (np.ndarray):  outcome/returns ndarray of dimensions T*N by 1.
        Z (np.ndarray):  covariates of dimensions T*N by p.
        T (int): number of time periods.
        N (int): number of observational units per time period.
        k (int): dimensionality of latent factors.
        p (int): dimensionality of the covariates.
        c (float):       scalar constant from theory; usually ~1.
        num_cpus (int): number of CPUs to use for parallel processing.
        demean (bool): optionally to demean the C_hat matrix before PCA.

    Returns:
        Gamma_beta_hat (np.ndarray): estimated loadings of dimensions p by k.
        F_hat (np.ndarray): estimated latent factors of dimensions T by k.
    '''
    # Figure out number cpus to use for outer and inner loops assuming we have at least 4
    assert(num_cpus >= 4)
    n_jobs_outer = int(num_cpus / 4)
    n_jobs_inner = 4

    def runForEachCharacteristic(j):
        # form indices
        minus_j = list(range(p))
        minus_j.remove(j)
        
        def runForEachTimePeriod(t):
            # form time start and end indices
            start_ob = int(t*N)
            last_ob  = int((t+1)*N)

            # form, for this rhs var j, this time periods LHS, target, and controls
            Y = R[start_ob:last_ob,:]
            D = Z[start_ob:last_ob,j].reshape(-1,1)
            X = Z[start_ob:last_ob,minus_j]

            # estimate c_{t,j}, i.e. target coef
            c_t_j = runDoubleSelectionLasso(Y, D, X, c)

            return c_t_j

        C_t_hat = Parallel(n_jobs=n_jobs_inner)(delayed(runForEachTimePeriod)(t) for t in range(T))
        
        return C_t_hat

    # Estimate C hat matrix
    C_hat = Parallel(n_jobs=n_jobs_outer)(delayed(runForEachCharacteristic)(j) for j in range(p))
    C_hat = np.array(C_hat).transpose()

    # Soft threshold C hat for DSLFM estimation of it
    C_hat_st = softThresholdCols(C_hat)

    # Demean C_hat for this version of the estimators
    C_hat_d = C_hat - np.mean(C_hat, axis=0)

    # Use PCA to decompose C_hat into estimated factors and loadings
    factors_hat, loadings_hat = fitBaiPCA(C_hat, T, k, p)
    Gamma_beta_hat = loadings_hat

    # Use PCA to decompose C_hat_d into estimated factors V and loadings \G_\b^d
    factors_v_hat, loadings_d_hat = fitBaiPCA(C_hat_d, T, k, p)
    Gamma_beta_d_hat = loadings_d_hat

    # Soft threshold Gamma beta hat and Gamma_beta_d_hat
    Gamma_beta_check = softThresholdRows(Gamma_beta_hat)
    Gamma_beta_d_check = softThresholdRows(Gamma_beta_d_hat)
    
    return C_hat_st, Gamma_beta_hat, factors_hat, Gamma_beta_check, factors_v_hat, Gamma_beta_d_check


In [38]:
def fitIPCA(Y: np.ndarray, X: np.ndarray, T: int, N: int, k: int, num_cpus: int):
    ''' Runs IPCA estimation procedure to return estimated loadings and factors.

    Args:
        Y (np.ndarray): single column of returns of dimension N*T by 1.
        X (np.ndarray): p columns of covariates with N*T rows.
        T (int): number of time periods.
        N (int): number of observational units per time period.
        k (int): dimensionality of latent factors.
        num_cpus (int): number of processors to use when working in parallel.

    Returns: (tuple)
        Gamma_beta_hat (np.ndarray): loadings of dimensions p by k.
        Factors_hat (np.ndarray): factors of dimensions T by k.
    '''
    # convert input objects to necessary object type and dimensions
    Y = pd.DataFrame(Y)
    X = pd.DataFrame(X)

    # add multiindex of integers time and asset to both LHS and RHS
    Y['time']  = np.repeat(np.arange(1,T+1), N)
    Y['asset'] = np.tile(np.arange(1,N+1), T)
    X['time']  = np.repeat(np.arange(1,T+1), N)
    X['asset'] = np.tile(np.arange(1,N+1), T)
    Y = Y.set_index(keys=['asset', 'time'], verify_integrity=True)
    Y = Y.squeeze() # convert to Series
    X = X.set_index(keys=['asset', 'time'], verify_integrity=True)

    # normalize RHS variables
    column_names = list(X.columns.values)
    for col in column_names:
        X = normalize_column(X, col)

    # fit IPCA
    ipca = InstrumentedPCA(n_factors=k, intercept=False, n_jobs=num_cpus)
    ipca = ipca.fit(X=X, y=Y, data_type='panel')

    # Form factors and loadings to return
    estimates = ipca.get_factors()
    Gamma_beta_hat  = estimates[0]
    Factors_hat   = estimates[1].transpose()
    
    return Gamma_beta_hat, Factors_hat


In [39]:
def calcAsympVar(Z: np.ndarray, ob_factor: np.ndarray, factors_hat: np.ndarray,
    gamma_beta_hat: np.ndarray, gamma_hat: np.ndarray, eta_hat: np.ndarray, 
    T: int, N: int, k: int, p: int):
    """ Calculate the asymptotic variance of the target parameter: risk premium of observable factor.
    
    Returns: (float) scalar estimated variance of risk premium of observable factor.
    """
    # Calculate the residuals from the time series OLS
    residuals = ob_factor - np.matmul(factors_hat, eta_hat)

    # Calculate the Z_t_j_jprime scalar
    Z_tjjp = np.zeros((T,p,p))

    for t in range(T):
        start_time_index = t*N
        end_time_index   = (t+1)*N
        for j in range(p):
            zitj_bar = np.mean(Z[start_time_index:end_time_index, j])
            Z_tjjp[t,j,:] = (9*Z*zitj_bar).mean(axis=0)

    Z_tjjp *= (N*T)**(-1)

    # Calculate the Pi_t scalar
    Pi = np.zeros((T,k,k))
    for t in range(T):
        Pi_t = 0
        for j in range(p):
            for jp in range(p):
                gamma_beta_jp = gamma_beta_hat[jp,:]
                gamma_beta_j  = gamma_beta_hat[j,:]
                Pi_t += gamma_beta_jp @ gamma_beta_j.T * Z_tjjp[t,j,jp]
        Pi[t] = Pi_t

    # Calc the components of asymp matrix
    Phi_11 = T**(-1) * (factors_hat.T @ residuals) * (residuals.T @ factors_hat)

    Phi_22 = np.zeros((k,k))
    for t in range(T):
        for tp in range(T):
            Phi_22 += Pi[t,:,:] @ factors_hat[t,:] * factors_hat[tp,:].T @ Pi[tp,:,:].T
    Phi_22 *= T**(-1)

    Phi_12 = np.zeros((k,k))
    for t in range(T):
        for tp in range(T):
            Phi_12 += factors_hat[t,:] * residuals[t] * factors_hat[tp,:].T @ Pi[tp,:,:].T
    Phi_12 *= T**(-1)

    # Calc design matrices
    A = (factors_hat.T @ factors_hat) / T

    Z_bar = Z.reshape(T,N,p).mean(axis=0)
    B = (gamma_beta_hat.T @ Z_bar.T @ Z_bar @ gamma_beta_hat) / N

    # Calculate the target variance
    sigma_g_2 = (gamma_hat.T @ np.linalg.inv(A) @ Phi_11 @ np.linalg.inv(A.T) @ gamma_hat
                    + eta_hat.T @ np.linalg.inv(B) @ Phi_22 @ np.linalg.inv(B.T) @ eta_hat
                    + gamma_hat.T @ np.linalg.inv(A) @ Phi_12 @ np.linalg.inv(B.T) @ eta_hat
                    + eta_hat.T @ np.linalg.inv(B) @ Phi_12.T @ np.linalg.inv(A.T) @ gamma_hat)

    return sigma_g_2


In [40]:
def calcGiglioVar(
    G_demeaned: np.ndarray, factors_hat: np.ndarray, eta_hat: np.ndarray, gamma_hat: np.ndarray):
    """ Calculate the asymptotic variance of the gamma_g estimator following Giglio 2021 eq. 11.

    Args:
        G_demeaned (np.ndarray): demeaned observable factors.
        factors_hat (np.ndarray): factor estimates, mean zero.
        eta_hat (np.ndarray): estimate of mapping from mean zero true latent factors to demeaned ob factor.
        gamma_hat (np.ndarray): estimate of the risk premium of true latent factors.
    """
    residuals = G_demeaned - factors_hat.dot(eta_hat)

    phi_11 = (factors_hat.T @ factors_hat) * (residuals.T @ residuals) / T
    phi_12 = (factors_hat.T @ factors_hat) * (residuals.T @ np.ones(T)) / T
    phi_22 = ((factors_hat.T @ np.ones(T)).reshape(-1,1) 
            @ (np.ones(T).T @ factors_hat).reshape(1,-1)) / T

    sigma_v = (factors_hat.T @ factors_hat) / T
    inv_sigma_v = np.linalg.inv(sigma_v)
    term1 = gamma_hat.T @ inv_sigma_v @ phi_11 @ inv_sigma_v @ gamma_hat
    term2 = gamma_hat.T @ inv_sigma_v @ phi_12 @ inv_sigma_v @ gamma_hat
    term3 = eta_hat.T @ phi_12.T @ inv_sigma_v @ gamma_hat
    term4 = eta_hat.T @ phi_22 @ eta_hat

    return term1 + term2 + term3 + term4


In [41]:
def calcEstimationErrors(param: np.ndarray, est: np.ndarray) -> tuple:
    ''' Calculate estimation errors for estimates across simulations.
    
    Args:
        param (np.ndarray): X by Y (by S).
        est (np.ndarray): X by Y by S.

    Where X and Y are arbitrary dimensions.

    Returns: (tuple): mse, bias2, and var.
    '''
    if param.ndim == 1:
        mse   = np.mean(np.square(est - param))
        bias2 = np.mean(np.square(np.mean(est) - np.mean(param)))
        var   = np.mean(np.square(est - np.mean(est)))
        return mse, bias2, var

    if param.ndim == 2:
        param = np.expand_dims(param, axis=-1)

    mean_param = np.mean(param, axis=2)
    mean_est   = np.expand_dims(np.mean(est, axis=2), axis=-1)

    mse   = np.mean(np.square(est - param))
    bias2 = np.mean(np.square(np.mean(est, axis=2) - mean_param))
    var   = np.mean(np.square(est - mean_est))

    return mse, bias2, var


In [ ]:
def calcCoverage(est: np.ndarray, var: np.ndarray, param: np.ndarray, alpha: float=0.05) -> float:
    """ Calculate the coverage of an inference procedure.
    
    Args:
        est (np.ndarray): vector of point estimates, dimensions S by 1.
        var (np.ndarray): vector of variances estimates, dimensions S by 1.
        param (np.ndarray): vector of true values, dimensions S by 1.
    
    Returns:
        float: Coverage probability.
    """
    critical_value = norm.ppf(1-alpha/2)

    upper_bound = est + critical_value * np.sqrt(var)
    lower_bound = est - critical_value * np.sqrt(var)

    coverage = np.mean((lower_bound <= param) & (param <= upper_bound))

    return coverage

In [10]:
def runSimulation(T: int, N: int, k: int, p: int, S: int, 
    R: np.ndarray, F: np.ndarray, G: np.ndarray, Z: np.ndarray, 
    Gamma_beta: np.ndarray, Gamma: np.ndarray, eta: np.ndarray,
    c: float, num_cpus: int):
    # Initialize objects to fill
    Factors_dslfm = np.zeros((T, k, S))
    C_hat_dslfm   = np.zeros((T, p, S))
    Beta_hat_dslfm = np.zeros((T*N, k, S))
    Gamma_beta_hat_dslfm = np.zeros((p, k, S))
    Gamma_beta_check_dslfm = np.zeros((p, k, S))
    Gamma_g_hat_dslfm = np.zeros((S))
    Gamma_g_var_dslfm = np.zeros((S))

    Factors_ipca = np.zeros((T, k, S))
    C_hat_ipca   = np.zeros((T, p, S))
    Beta_hat_ipca = np.zeros((T*N, k, S))
    Gamma_beta_ipca = np.zeros((p, k, S))
    Gamma_g_hat_ipca = np.zeros((S))
    Gamma_g_var_ipca = np.zeros((S))

    Factors_giglio = np.zeros((T, k, S))
    Factors_f_giglio = np.zeros((T, k, S))
    Beta_hat_giglio = np.zeros((N, k, S))
    Gamma_g_hat_giglio = np.zeros((S))
    Gamma_g_var_giglio = np.zeros((S))

    C_0 = np.zeros((T, p, S))
    Beta_0 = np.zeros((T*N, k, S))
    Gamma_g_0 = np.zeros((S))

    # Run simulation
    for s in range(S):
        # Monitor progress
        print(s)

        # Calculate needed random variables
        return_matrix = R[:,:,s].reshape(T,N)
        giglio_return_matrix = return_matrix - np.mean(return_matrix, axis=0)
        R_demeaned = return_matrix - return_matrix.mean(axis=0)
        R_bar = (return_matrix - R_demeaned).mean(axis=0).reshape(-1, 1)
        G_demeaned = G[:,:,s]-np.mean(G[:,:,s])
        Z_bar = Z[:,:,s].reshape(T,N,p).mean(axis=0)

        # Calculate true target parameters
        Gamma_g_0[s] = eta.reshape(-1,1).T.dot(Gamma[:,:,s])[0][0]
        C_0[:,:,s] = F[:,:,s] @ Gamma_beta.T
        Beta_0[:,:,s] = Z[:,:,s] @ Gamma_beta

        # DSLFM estimation
        c_hat_st, gamma_beta_dslfm, factors_dslfm, gamma_beta_check_dslfm, factors_v_dslfm, gamma_beta_d_check_dslfm = fitDSLFM(
            R[:,:,s], Z[:,:,s], T, N, k, p, c, num_cpus)
        C_hat_dslfm[:,:,s] = c_hat_st
        Beta_hat_dslfm[:,:,s] = Z[:,:,s] @ gamma_beta_check_dslfm
        beta_hat_bar = np.matmul(Z_bar, gamma_beta_d_check_dslfm)
        gamma_hat_dslfm = runOLS(R_bar, beta_hat_bar).reshape(-1)
        eta_hat_dslfm   = runOLS(G_demeaned, factors_v_dslfm)
        Gamma_g_hat_dslfm[s] = np.dot(eta_hat_dslfm.T, gamma_hat_dslfm)[0]
        Gamma_g_var_dslfm[s] = calcAsympVar(Z[:,:,s], G_demeaned, factors_v_dslfm, 
            gamma_beta_dslfm, gamma_hat_dslfm, eta_hat_dslfm, 
            T, N, p)
        Factors_dslfm[:,:,s] = factors_dslfm
        Gamma_beta_hat_dslfm[:,:,s] = gamma_beta_dslfm
        Gamma_beta_check_dslfm[:, :, s] = gamma_beta_check_dslfm

        # IPCA estimation
        gamma_beta_ipca, factors_ipca = fitIPCA(R[:,:,s], Z[:,:,s], T, N, k, num_cpus)
        C_hat_ipca[:,:,s] = factors_ipca @ gamma_beta_ipca.T
        Beta_hat_ipca[:,:,s] = Z[:,:,s] @ gamma_beta_ipca
        gamma_hat_ipca = np.mean(factors_ipca, axis=0)
        factors_v_ipca = factors_ipca-gamma_hat_ipca
        eta_hat_ipca   = runOLS(G_demeaned, factors_v_ipca)
        Gamma_g_hat_ipca[s] = np.dot(eta_hat_ipca.T, gamma_hat_ipca)[0]
        Gamma_g_var_ipca[s] = calcAsympVar(G_demeaned, factors_v_ipca, gamma_hat_ipca, 
            eta_hat_ipca, T)
        Factors_ipca[:,:,s] = factors_ipca
        Gamma_beta_ipca[:,:,s] = gamma_beta_ipca

        # Giglio Estimation
        factors_giglio, beta_giglio = fitBaiPCA(giglio_return_matrix, T, k, N)
        Beta_hat_giglio[:,:,s] = beta_giglio
        gamma_hat_giglio = runOLS(R_bar, beta_giglio)
        eta_hat_giglio = runOLS(G_demeaned, factors_giglio)
        Factors_giglio[:,:,s] = factors_giglio
        Factors_f_giglio[:,:,s] = factors_giglio + gamma_hat_giglio.reshape(-1)
        Gamma_g_hat_giglio[s] = eta_hat_giglio.T.dot(gamma_hat_giglio)
        Gamma_g_var_giglio[s] = calcGiglioVar(G_demeaned, factors_giglio, eta_hat_giglio, gamma_hat_giglio)

    return (C_hat_dslfm, Beta_hat_dslfm, Factors_dslfm, Gamma_beta_check_dslfm, Gamma_beta_hat_dslfm, 
        Gamma_g_hat_dslfm, Gamma_g_var_dslfm,
        C_hat_ipca, Beta_hat_ipca, Factors_ipca, Gamma_beta_ipca, Gamma_g_hat_ipca,
        Beta_hat_giglio, Factors_f_giglio, Gamma_g_hat_giglio, Gamma_g_var_giglio,
        C_0, Beta_0, Gamma_g_0)



In [ ]:
if __name__ == "__main__":
    # set simulation parameters
    PANEL_WEEKLY_IN_FP = '../data/clean/panel_weekly.pkl'
    ASSET_UNI_IN_FP    = '../data/clean/asset_universe_dict.pickle'
    LHS_COL            = 'r_ex_tp7'
    NUM_CPUS           = 20
    T                  = 100
    N                  = 200
    k                  = 3
    p                  = 10
    SPARSE_S           = int(p/5)
    S                  = 3
    FIT_ALPHA          = False
    ETA                = np.array([1]+list(np.zeros(k-1)))
    SIGMA_F_2          = 0.01
    SIGMA_Z_2          = 0.01
    SIGMA_G_2          = 0.01
    SIGMA_E_2          = 0.01
    C                  = 1

    # # TEMP TODO LOOP OVER NOISE PARAMS TO GEN RESULTS ACROSS
    # for SIGMA_F_2 in [0.5, 0.1, 0.05]:
    #     for SIGMA_Z_2 in [0.5, 0.1, 0.05]:
    #         for SIGMA_G_2 in [0.1, 0.05, 0.005]:
    #             for SIGMA_E_2 in [0.5, 0.1, 0.05]:
                    
    # read in data
    with open(ASSET_UNI_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    panel_df = pd.read_pickle(PANEL_WEEKLY_IN_FP)

    # fit ipca in the panel to obtain factors, loadings, and r^2_total
    ipca_factors, ipca_loadings, r2_total_emp, selected_columns = fitIPCAInPanelForEstimatedFactorsAndLoadings(
        panel_df, LHS_COL, p, k, FIT_ALPHA, NUM_CPUS
    )

    # build the data
    R, Gamma_beta, F, G, Z, Gamma, e, e_g = DGP(T, N, k, p, SPARSE_S, S, ETA,
        SIGMA_F_2, SIGMA_Z_2, SIGMA_G_2, SIGMA_E_2,
        ipca_factors, ipca_loadings, r2_total_emp, selected_columns, panel_df, NUM_CPUS)
    
    # run the simulation
    (C_hat_dslfm, Beta_hat_dslfm, Factors_dslfm, Gamma_beta_check_dslfm, Gamma_beta_dslfm, 
        Gamma_g_hat_dslfm, Gamma_g_var_dslfm,
        C_hat_ipca, Beta_hat_ipca, Factors_ipca, Gamma_beta_ipca, Gamma_g_hat_ipca,
        Beta_hat_giglio, Factors_f_giglio, Gamma_g_hat_giglio, Gamma_g_var_giglio,
        C_0, Beta_0, Gamma_g_0) = runSimulation(T, N, k, p, S, R, F, G, Z, Gamma_beta, Gamma, ETA, C, NUM_CPUS)
    
    # Calculate estimation errors
    mse_c_dslfm, bias2_c_dslfm, var_c_dslfm = calcEstimationErrors(C_0, C_hat_dslfm)
    mse_factors_dslfm, bias2_factors_dslfm, var_factors_dslfm = calcEstimationErrors(F, Factors_dslfm)
    mse_gamma_b_dslfm, bias2_gamma_b_dslfm, var_gamma_b_dslfm = calcEstimationErrors(Gamma_beta, Gamma_beta_check_dslfm)
    mse_beta_dslfm, bias2_beta_dslfm, var_beta_dslfm = calcEstimationErrors(Beta_0.reshape(T,N,k,S).mean(axis=0), 
                                                                            Beta_hat_dslfm.reshape(T,N,k,S).mean(axis=0))
    mse_gamma_g_dslfm, bias2_gamma_g_dslfm, var_gamma_g_dslfm = calcEstimationErrors(Gamma_g_0, Gamma_g_hat_dslfm)
    
    mse_c_ipca, bias2_c_ipca, var_c_ipca = calcEstimationErrors(C_0, C_hat_ipca)
    mse_factors_ipca, bias2_factors_ipca, var_factors_ipca = calcEstimationErrors(F, Factors_ipca)
    mse_gamma_b_ipca, bias2_gamma_b_ipca, var_gamma_b_ipca = calcEstimationErrors(Gamma_beta, Gamma_beta_ipca)
    mse_beta_ipca, bias2_beta_ipca, var_beta_ipca = calcEstimationErrors(Beta_0.reshape(T,N,k,S).mean(axis=0), 
                                                                        Beta_hat_ipca.reshape(T,N,k,S).mean(axis=0))
    mse_gamma_g_ipca, bias2_gamma_g_ipca, var_gamma_g_ipca = calcEstimationErrors(Gamma_g_0, Gamma_g_hat_ipca)

    mse_factors_giglio, bias2_factors_giglio, var_factors_giglio = calcEstimationErrors(F, Factors_f_giglio)
    mse_beta_giglio, bias2_beta_giglio, var_beta_giglio = calcEstimationErrors(Beta_0.reshape(T,N,k,S).mean(axis=0), 
                                                                            Beta_hat_giglio)
    mse_gamma_g_giglio, bias2_gamma_g_giglio, var_gamma_g_giglio = calcEstimationErrors(Gamma_g_0, Gamma_g_hat_giglio)

    # Calculate coverage
    cov90_dslfm = calcCoverage(Gamma_g_hat_dslfm, Gamma_g_var_dslfm, Gamma_g_0, alpha=0.1)
    cov90_giglio = calcCoverage(Gamma_g_hat_giglio, Gamma_g_var_giglio, Gamma_g_0, alpha=0.1)
    cov95_dslfm = calcCoverage(Gamma_g_hat_dslfm, Gamma_g_var_dslfm, Gamma_g_0, alpha=0.05)
    cov95_giglio = calcCoverage(Gamma_g_hat_giglio, Gamma_g_var_giglio, Gamma_g_0, alpha=0.05)

    # Save results
    timestr = time.strftime("%Y%m%d_%H%M%S")
    results_df = pd.DataFrame(data = {'timestamp': [timestr, timestr, timestr],
        'S': [S, S, S], 'T': [T, T, T], 'N': [N, N, N], 'p': [p, p, p], 'k': [k, k, k],
        'sparse_s': [SPARSE_S, SPARSE_S, SPARSE_S], 'c': [C, C, C], 
        'sigma_f_2': [SIGMA_F_2, SIGMA_F_2, SIGMA_F_2], 'sigma_z_2': [SIGMA_Z_2, SIGMA_Z_2, SIGMA_Z_2], 
        'sigma_g_2': [SIGMA_G_2, SIGMA_G_2, SIGMA_G_2], 'sigma_e_2': [SIGMA_E_2, SIGMA_E_2, SIGMA_E_2],
        'method': ['dslfm', 'ipca', 'giglio'],
        'mse_c': [mse_c_dslfm, mse_c_ipca, 0],
        'bias2_c': [bias2_c_dslfm, bias2_c_ipca, 0],
        'var_c': [var_c_dslfm, var_c_ipca, 0],
        'mse_f': [mse_factors_dslfm, mse_factors_ipca, mse_factors_giglio], 
        'bias2_f': [bias2_factors_dslfm, bias2_factors_ipca, bias2_factors_giglio], 
        'var_f': [var_factors_dslfm, var_factors_ipca, var_factors_giglio],
        'mse_g_b': [mse_gamma_b_dslfm, mse_gamma_b_ipca, 0], 
        'bias2_g_b': [bias2_gamma_b_dslfm, bias2_gamma_b_ipca, 0], 
        'var_g_b': [var_gamma_b_dslfm, var_gamma_b_ipca, 0], 
        'mse_b': [mse_beta_dslfm, mse_beta_ipca, mse_beta_giglio],        
        'bias2_b': [bias2_beta_dslfm, bias2_beta_ipca, bias2_beta_giglio],
        'var_b': [var_beta_dslfm, var_beta_ipca, var_beta_giglio],
        'mse_g_g': [mse_gamma_g_dslfm, mse_gamma_g_ipca, mse_gamma_g_giglio], 
        'bias2_g_g': [bias2_gamma_g_dslfm, bias2_gamma_g_ipca, bias2_gamma_g_giglio], 
        'var_g_g': [var_gamma_g_dslfm, var_gamma_g_ipca, var_gamma_g_giglio], 
        'cov90_g_g': [cov90_dslfm, 0, cov90_giglio],
        'cov95_g_g': [cov95_dslfm, 0, cov95_giglio]})
    results_df.to_csv('../output/high_dim_fm/sim_'+timestr+'.csv', index=False)
